# Structured data classification with FeatureSpace

In [1]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from sklearn.metrics import classification_report

In [2]:
file_url = "http://storage.googleapis.com/download.tensorflow.org/data/heart.csv"
dataframe = pd.read_csv(file_url)

In [3]:
print(dataframe.shape)

(303, 14)


In [4]:
dataframe.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0


In [6]:
val_dataframe = dataframe.sample(frac=0.2, random_state=1337)
train_dataframe = dataframe.drop(val_dataframe.index)

print(
    "Using %d samples for training and %d for validation"
    % (len(train_dataframe), len(val_dataframe))
)

Using 242 samples for training and 61 for validation


In [7]:
def dataframe_to_dataset(dataframe):
    dataframe = dataframe.copy()
    labels = dataframe.pop("target")
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    ds = ds.shuffle(buffer_size=len(dataframe))
    return ds


train_ds = dataframe_to_dataset(train_dataframe)
val_ds = dataframe_to_dataset(val_dataframe)

In [8]:
for x, y in train_ds.take(1):
    print("Input:", x)
    print("Target:", y)

Input: {'age': <tf.Tensor: shape=(), dtype=int64, numpy=42>, 'sex': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'cp': <tf.Tensor: shape=(), dtype=int64, numpy=3>, 'trestbps': <tf.Tensor: shape=(), dtype=int64, numpy=120>, 'chol': <tf.Tensor: shape=(), dtype=int64, numpy=209>, 'fbs': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'restecg': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'thalach': <tf.Tensor: shape=(), dtype=int64, numpy=173>, 'exang': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'oldpeak': <tf.Tensor: shape=(), dtype=float64, numpy=0.0>, 'slope': <tf.Tensor: shape=(), dtype=int64, numpy=2>, 'ca': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'thal': <tf.Tensor: shape=(), dtype=string, numpy=b'normal'>}
Target: tf.Tensor(0, shape=(), dtype=int64)


In [9]:
train_ds = train_ds.batch(32)
val_ds = val_ds.batch(32)



In [10]:
from keras.utils import FeatureSpace

feature_space = FeatureSpace(
    features={
        # Categorical features encoded as integers
        "sex": "integer_categorical",
        "cp": "integer_categorical",
        "fbs": "integer_categorical",
        "restecg": "integer_categorical",
        "exang": "integer_categorical",
        "ca": "integer_categorical",
        # Categorical feature encoded as string
        "thal": "string_categorical",
        # Numerical features to discretize
        "age": "float_discretized",
        # Numerical features to normalize
        "trestbps": "float_normalized",
        "chol": "float_normalized",
        "thalach": "float_normalized",
        "oldpeak": "float_normalized",
        "slope": "float_normalized",
    },
    # We create additional features by hashing
    # value co-occurrences for the
    # following groups of categorical features.
    crosses=[("sex", "age"), ("thal", "ca")],
    # The hashing space for these co-occurrences
    # wil be 32-dimensional.
    crossing_dim=32,
    # Our utility will one-hot encode all categorical
    # features and concat all features into a single
    # vector (one vector per sample).
    output_mode="concat",
)

# Details

- num_oov_indices is saying that all the categories are present on the training data i.e. =Male/Female, but if there are possible unseen categories you would need to set this to 1

- num_bins => # of parts the age (example) would be broken into by the model, the higher the finer details and lower is less but faster

In [11]:
feature_space = FeatureSpace(
    features={
        # Categorical features encoded as integers
        "sex": FeatureSpace.integer_categorical(num_oov_indices=0),
        "cp": FeatureSpace.integer_categorical(num_oov_indices=0),
        "fbs": FeatureSpace.integer_categorical(num_oov_indices=0),
        "restecg": FeatureSpace.integer_categorical(num_oov_indices=0),
        "exang": FeatureSpace.integer_categorical(num_oov_indices=0),
        "ca": FeatureSpace.integer_categorical(num_oov_indices=0),
        # Categorical feature encoded as string
        "thal": FeatureSpace.string_categorical(num_oov_indices=0),
        # Numerical features to normalize
        "age": FeatureSpace.float_discretized(num_bins=30),
        # Numerical features to normalize
        "trestbps": FeatureSpace.float_normalized(),
        "chol": FeatureSpace.float_normalized(),
        "thalach": FeatureSpace.float_normalized(),
        "oldpeak": FeatureSpace.float_normalized(),
        "slope": FeatureSpace.float_normalized(),
    },
    # Specify feature cross with a custom crossing dim.
    crosses=[
        FeatureSpace.cross(feature_names=("sex", "age"), crossing_dim=64),
        FeatureSpace.cross(
            feature_names=("thal", "ca"),
            crossing_dim=16,
        ),
    ],
    output_mode="concat",
)

train_ds_with_no_labels = train_ds.map(lambda x, _: x)

basically says that the labels would be ignored and just return the features on the train_ds dataset

In [12]:
train_ds_with_no_labels = train_ds.map(lambda x, _: x)
feature_space.adapt(train_ds_with_no_labels)

In [13]:
for x, _ in train_ds.take(1):
    preprocessed_x = feature_space(x)
    print("preprocessed_x.shape:", preprocessed_x.shape)
    print("preprocessed_x.dtype:", preprocessed_x.dtype)

preprocessed_x.shape: (32, 138)
preprocessed_x.dtype: <dtype: 'float32'>


# preprocessing
Let's break it down step by step:

train_ds.map(lambda x, y: (feature_space(x), y), num_parallel_calls=tf.data.AUTOTUNE):

train_ds is your original training dataset, presumably containing pairs of input data (x) and labels (y).
The map function is used to apply a lambda function to each element of train_ds.
The lambda function takes two input arguments, x and y, where x is the input data, and y is the label. The lambda function performs two actions:
feature_space(x) is called to transform the input data x using the feature_space. This likely involves feature preprocessing, encoding, and any necessary transformations defined in your feature space.
The label y is left unchanged.
So, each element in the original dataset is transformed to contain the preprocessed input data and the original label. The resulting dataset contains pairs of (preprocessed input data, label).
preprocessed_train_ds = preprocessed_train_ds.prefetch(tf.data.AUTOTUNE):

This line of code applies the prefetch method to the preprocessed_train_ds dataset.
prefetch is used to improve data loading performance during training. It prefetches data from the dataset so that it is ready and available when needed by the model during training.
tf.data.AUTOTUNE is used as an argument to prefetch. This value allows TensorFlow to automatically adjust the prefetch buffer size based on the available resources and can help optimize data loading performance.
In summary, the provided code first preprocesses the input data in the training dataset using the feature_space and retains the original labels. Then, it sets up data prefetching to improve training performance by making sure data is readily available when needed during training. This is a common practice in deep learning to ensure efficient data pipeline processing.

In [14]:
preprocessed_train_ds = train_ds.map(
    lambda x, y: (feature_space(x), y), num_parallel_calls=tf.data.AUTOTUNE
)
preprocessed_train_ds = preprocessed_train_ds.prefetch(tf.data.AUTOTUNE)

preprocessed_val_ds = val_ds.map(
    lambda x, y: (feature_space(x), y), num_parallel_calls=tf.data.AUTOTUNE
)
preprocessed_val_ds = preprocessed_val_ds.prefetch(tf.data.AUTOTUNE)

In [15]:
dict_inputs = feature_space.get_inputs()
encoded_features = feature_space.get_encoded_features()

x = keras.layers.Dense(32, activation="relu")(encoded_features)
x = keras.layers.Dropout(0.5)(x)
predictions = keras.layers.Dense(1, activation="sigmoid")(x)

training_model = keras.Model(inputs=encoded_features, outputs=predictions)
training_model.compile(
    optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"]
)

inference_model = keras.Model(inputs=dict_inputs, outputs=predictions)

In [16]:
training_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 138)]             0         
                                                                 
 dense (Dense)               (None, 32)                4448      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 4481 (17.50 KB)
Trainable params: 4481 (17.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
inference_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 age (InputLayer)            [(None, 1)]                  0         []                            
                                                                                                  
 ca (InputLayer)             [(None, 1)]                  0         []                            
                                                                                                  
 sex (InputLayer)            [(None, 1)]                  0         []                            
                                                                                                  
 thal (InputLayer)           [(None, 1)]                  0         []                            
                                                                                            

In [18]:
training_model.fit(
    preprocessed_train_ds, epochs=20, validation_data=preprocessed_val_ds, verbose=2
)

Epoch 1/20
8/8 - 1s - loss: 0.7788 - accuracy: 0.4174 - val_loss: 0.7284 - val_accuracy: 0.4754 - 1s/epoch - 166ms/step
Epoch 2/20
8/8 - 0s - loss: 0.7192 - accuracy: 0.4959 - val_loss: 0.6698 - val_accuracy: 0.6230 - 288ms/epoch - 36ms/step
Epoch 3/20
8/8 - 0s - loss: 0.6594 - accuracy: 0.6240 - val_loss: 0.6217 - val_accuracy: 0.7541 - 278ms/epoch - 35ms/step
Epoch 4/20
8/8 - 0s - loss: 0.6041 - accuracy: 0.7025 - val_loss: 0.5802 - val_accuracy: 0.7705 - 254ms/epoch - 32ms/step
Epoch 5/20
8/8 - 0s - loss: 0.5706 - accuracy: 0.7355 - val_loss: 0.5468 - val_accuracy: 0.7869 - 272ms/epoch - 34ms/step
Epoch 6/20
8/8 - 0s - loss: 0.5545 - accuracy: 0.7273 - val_loss: 0.5182 - val_accuracy: 0.7705 - 257ms/epoch - 32ms/step
Epoch 7/20
8/8 - 0s - loss: 0.5058 - accuracy: 0.7686 - val_loss: 0.4932 - val_accuracy: 0.7869 - 266ms/epoch - 33ms/step
Epoch 8/20
8/8 - 0s - loss: 0.4770 - accuracy: 0.8058 - val_loss: 0.4718 - val_accuracy: 0.8033 - 255ms/epoch - 32ms/step
Epoch 9/20
8/8 - 0s - loss

In [19]:
sample = {
    "age": 60,
    "sex": 1,
    "cp": 1,
    "trestbps": 145,
    "chol": 233,
    "fbs": 1,
    "restecg": 2,
    "thalach": 150,
    "exang": 0,
    "oldpeak": 2.3,
    "slope": 3,
    "ca": 0,
    "thal": "fixed",
}

input_dict = {name: tf.convert_to_tensor([value]) for name, value in sample.items()}
predictions = inference_model.predict(input_dict)

print(
    f"This particular patient had a {100 * predictions[0][0]:.2f}% probability "
    "of having a heart disease, as evaluated by our model."
)

1/1 [==============================] - 1s 561ms/step
This particular patient had a 43.29% probability of having a heart disease, as evaluated by our model.


In [20]:
predictions = training_model.predict(preprocessed_val_ds)


print(
    f"This particular patient had a {100 * predictions[3][0]:.2f}% probability "
    "of having a heart disease, as evaluated by our model."
)
print(predictions) 

2/2 [==============================] - 0s 3ms/step
This particular patient had a 19.33% probability of having a heart disease, as evaluated by our model.
[[0.01430352]
 [0.32297876]
 [0.01462181]
 [0.19328482]
 [0.01688223]
 [0.02799104]
 [0.629591  ]
 [0.51616323]
 [0.12140415]
 [0.06395338]
 [0.02236175]
 [0.11545574]
 [0.02728997]
 [0.16065489]
 [0.07310157]
 [0.6229378 ]
 [0.10275718]
 [0.04237199]
 [0.27192584]
 [0.93540454]
 [0.02507029]
 [0.06665181]
 [0.7790163 ]
 [0.60965836]
 [0.8018422 ]
 [0.7862586 ]
 [0.04911359]
 [0.19362806]
 [0.06521329]
 [0.03892029]
 [0.5636754 ]
 [0.20021905]
 [0.04432646]
 [0.38866776]
 [0.7595991 ]
 [0.5844091 ]
 [0.10397639]
 [0.2349407 ]
 [0.1205594 ]
 [0.16104293]
 [0.11144549]
 [0.04064574]
 [0.6950112 ]
 [0.18230423]
 [0.6296398 ]
 [0.10741557]
 [0.12113087]
 [0.06441261]
 [0.08390397]
 [0.16959573]
 [0.05613362]
 [0.77709275]
 [0.2659073 ]
 [0.08648174]
 [0.06722599]
 [0.19115297]
 [0.10056996]
 [0.86188906]
 [0.86211336]
 [0.02416883]
 [0.02

In [63]:
import numpy as np

# Create a list to store the expected and predicted values
expected_values = []
predicted_values = []

# Define a function to convert integer labels to strings
def label_to_string(label):
    return "1" if label == 1 else "0"

# Loop through the preprocessed_val_ds
for features, labels in preprocessed_val_ds.take(3):  # Take the first 2 records
    # Make predictions using the trained model
    print(features.shape)
    print(labels.shape)
    predictions = training_model.predict(features)
    print(f"preds: {predictions}")

    # Append the expected and predicted values to the respective lists
    expected_values.extend([label_to_string(int(label)) for label in labels])
    # print(f"exp = {expected_values}")
    predicted_values.extend([str(int(round(float(pred[0])))) for pred in predictions])
    # print(f"pre = {predicted_values}")


# Display the expected and predicted values
display_predictions(expected_values, predicted_values)



(32, 138)
(32,)
1/1 [==============================] - 0s 38ms/step
preds: [[0.00834814]
 [0.8983119 ]
 [0.09383976]
 [0.08347861]
 [0.13623661]
 [0.104359  ]
 [0.06022298]
 [0.00650315]
 [0.01403308]
 [0.5041339 ]
 [0.28269184]
 [0.05247024]
 [0.02318082]
 [0.9419776 ]
 [0.553237  ]
 [0.00761372]
 [0.11790613]
 [0.44192562]
 [0.86034477]
 [0.08350082]
 [0.01441471]
 [0.02224759]
 [0.02037767]
 [0.03341683]
 [0.8820038 ]
 [0.15430942]
 [0.00802119]
 [0.1487403 ]
 [0.06930605]
 [0.0322541 ]
 [0.21358001]
 [0.02140102]]
(29, 138)
(29,)
1/1 [==============================] - 0s 29ms/step
preds: [[0.05329962]
 [0.10920398]
 [0.55480707]
 [0.7658504 ]
 [0.0959092 ]
 [0.79075134]
 [0.35983476]
 [0.7902093 ]
 [0.02788934]
 [0.03813506]
 [0.10183028]
 [0.82336724]
 [0.15050347]
 [0.02914582]
 [0.15267485]
 [0.8994114 ]
 [0.00672916]
 [0.7374681 ]
 [0.5842109 ]
 [0.08742587]
 [0.6307158 ]
 [0.08239064]
 [0.23159355]
 [0.10918142]
 [0.06973504]
 [0.01117723]
 [0.8372309 ]
 [0.13564537]
 [0.01180

In [73]:
val_dataframe.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
96,41,1,3,112,250,0,0,179,0,0.0,1,0,normal,0
142,67,1,4,100,299,0,2,125,1,0.9,2,2,normal,1
80,51,1,3,94,227,0,0,154,1,0.0,1,1,reversible,0
67,41,1,2,135,203,0,0,132,0,0.0,2,0,fixed,0
188,41,0,2,126,306,0,0,163,0,0.0,1,0,normal,0


In [76]:
#2 recs from val:
recs2 = val_dataframe.iloc[:2].copy()
recs2.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
96,41,1,3,112,250,0,0,179,0,0.0,1,0,normal,0
142,67,1,4,100,299,0,2,125,1,0.9,2,2,normal,1


In [98]:
def dataframe_to_dataset2(dataframe):
    dataframe = dataframe.copy()
    labels = dataframe.pop("target")
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    ds = ds.shuffle(buffer_size=len(dataframe))
    return ds

# rec2_ds_with_no_labels = rec2_ds.map(lambda x, _: x)
# feature_space.adapt(rec2_ds_with_no_labels)

rec2_ds = dataframe_to_dataset2(recs2)
rec2_ds = rec2_ds.batch(32)

preprocessed_rec2_ds = rec2_ds.map(
    lambda x, y: (feature_space(x), y), num_parallel_calls=tf.data.AUTOTUNE
)
preprocessed_rec2_ds = preprocessed_rec2_ds.prefetch(tf.data.AUTOTUNE)

In [95]:
print(rec2_ds)

<_TensorSliceDataset element_spec=({'age': TensorSpec(shape=(), dtype=tf.int64, name=None), 'sex': TensorSpec(shape=(), dtype=tf.int64, name=None), 'cp': TensorSpec(shape=(), dtype=tf.int64, name=None), 'trestbps': TensorSpec(shape=(), dtype=tf.int64, name=None), 'chol': TensorSpec(shape=(), dtype=tf.int64, name=None), 'fbs': TensorSpec(shape=(), dtype=tf.int64, name=None), 'restecg': TensorSpec(shape=(), dtype=tf.int64, name=None), 'thalach': TensorSpec(shape=(), dtype=tf.int64, name=None), 'exang': TensorSpec(shape=(), dtype=tf.int64, name=None), 'oldpeak': TensorSpec(shape=(), dtype=tf.float64, name=None), 'slope': TensorSpec(shape=(), dtype=tf.int64, name=None), 'ca': TensorSpec(shape=(), dtype=tf.int64, name=None), 'thal': TensorSpec(shape=(), dtype=tf.string, name=None)}, TensorSpec(shape=(), dtype=tf.int64, name=None))>


In [104]:
for x, label in preprocessed_rec2_ds.take(2):
    # preprocessed_x = feature_space(x)
    # print("preprocessed_x.shape:", preprocessed_x.shape)
    # print("preprocessed_x.dtype:", preprocessed_x.dtype)
    predictions = training_model.predict(x)
    print(predictions[0])
    print(predictions[1])
    print(x.shape)
    print(label)

# for x, label in preprocessed_train_ds.take(1):
#     # preprocessed_x = feature_space(x)
#     # print("preprocessed_x.shape:", preprocessed_x.shape)
#     # print("preprocessed_x.dtype:", preprocessed_x.dtype)
#     # predictions = training_model.predict(x)
#     # print(predictions[0])
#     print(x.shape)
#     print(label)

1/1 [==============================] - 0s 27ms/step
[0.3071746]
[0.05420016]
(2, 138)
tf.Tensor([1 0], shape=(2,), dtype=int64)


In [105]:

dict_inputs = feature_space.get_inputs()
encoded_features = feature_space.get_encoded_features()

sq_model = keras.Sequential()
sq_model.add(keras.layers.Dense(32, activation="relu", input_shape=encoded_features.shape[1:]))
sq_model.add(keras.layers.Dropout(0.5))
sq_model.add(keras.layers.Dense(1, activation="sigmoid"))

sq_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])


In [106]:
sq_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 32)                4448      
                                                                 
 dropout_2 (Dropout)         (None, 32)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 4481 (17.50 KB)
Trainable params: 4481 (17.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [108]:
sq_model.fit(
    preprocessed_train_ds, epochs=20, validation_data=preprocessed_val_ds, verbose=2
)

Epoch 1/20
8/8 - 0s - loss: 0.4060 - accuracy: 0.8264 - val_loss: 0.3716 - val_accuracy: 0.8033 - 457ms/epoch - 57ms/step
Epoch 2/20
8/8 - 0s - loss: 0.3963 - accuracy: 0.8264 - val_loss: 0.3664 - val_accuracy: 0.8197 - 415ms/epoch - 52ms/step
Epoch 3/20
8/8 - 0s - loss: 0.3466 - accuracy: 0.8388 - val_loss: 0.3628 - val_accuracy: 0.8197 - 468ms/epoch - 58ms/step
Epoch 4/20
8/8 - 0s - loss: 0.3551 - accuracy: 0.8430 - val_loss: 0.3607 - val_accuracy: 0.8197 - 445ms/epoch - 56ms/step
Epoch 5/20
8/8 - 0s - loss: 0.3503 - accuracy: 0.8430 - val_loss: 0.3587 - val_accuracy: 0.8525 - 471ms/epoch - 59ms/step
Epoch 6/20
8/8 - 1s - loss: 0.3365 - accuracy: 0.8678 - val_loss: 0.3579 - val_accuracy: 0.8525 - 568ms/epoch - 71ms/step
Epoch 7/20
8/8 - 0s - loss: 0.3217 - accuracy: 0.8512 - val_loss: 0.3572 - val_accuracy: 0.8525 - 443ms/epoch - 55ms/step
Epoch 8/20
8/8 - 0s - loss: 0.3141 - accuracy: 0.8843 - val_loss: 0.3587 - val_accuracy: 0.8525 - 465ms/epoch - 58ms/step
Epoch 9/20
8/8 - 0s - lo

In [133]:
# for x, label in preprocessed_rec2_ds.take(2):
#     # preprocessed_x = feature_space(x)
#     # print("preprocessed_x.shape:", preprocessed_x.shape)
#     # print("preprocessed_x.dtype:", preprocessed_x.dtype)
#     predictions = sq_model.predict(x)
#     print(predictions[0])
#     print(predictions[1])
#     print(x.shape)
#     print(label)

m_predictions = training_model.predict(preprocessed_val_ds)

predictions = sq_model.predict(preprocessed_val_ds)
# print(predictions)
# label_values = [label for _, label in preprocessed_val_ds.as_numpy_iterator()]
# # print(label_values)
# Initialize an empty list to collect labels
label_values = []

# Iterate through preprocessed_val_ds to collect labels
for _, labels in preprocessed_val_ds.as_numpy_iterator():
    label_values.extend(labels)

# Convert the collected labels to a NumPy array
label_values = np.array(label_values)



2/2 [==============================] - 1s 7ms/step


In [142]:
predictions = (predictions > 0.3).astype(int).reshape(-1,)
m_predictions = (m_predictions > 0.3).astype(int).reshape(-1,)

In [143]:
print(label_values)

[0 0 1 1 1 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 1 1 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0]


In [144]:
print(classification_report(label_values, predictions))

print(classification_report(label_values, m_predictions))

              precision    recall  f1-score   support

           0       0.77      0.72      0.75        47
           1       0.24      0.29      0.26        14

    accuracy                           0.62        61
   macro avg       0.50      0.50      0.50        61
weighted avg       0.65      0.62      0.63        61

              precision    recall  f1-score   support

           0       0.78      0.74      0.76        47
           1       0.25      0.29      0.27        14

    accuracy                           0.64        61
   macro avg       0.51      0.52      0.51        61
weighted avg       0.66      0.64      0.65        61



In [51]:
sample = {
    "age": 60,
    "sex": 1,
    "cp": 1,
    "trestbps": 145,
    "chol": 233,
    "fbs": 1,
    "restecg": 2,
    "thalach": 150,
    "exang": 0,
    "oldpeak": 2.3,
    "slope": 3,
    "ca": 0,
    "thal": "fixed",
}

input_dict = {name: tf.convert_to_tensor([value]) for name, value in sample.items()}
print(input_dict["age"].shape)


(1,)
